In [11]:
from photosynthesis_metrics import vif_p
import torch
x = torch.rand(1, 3, 256, 256).cuda()
y = torch.rand(1, 3, 256, 256).cuda()
# torch.isnan(vif_p(x, x))
score = vif_p(x, x)
score

tensor([1.0000], device='cuda:0')

In [30]:
# measure = torch.tensor([0.9345, 1])
measure = torch.tensor(0.9345)
# measure.dim()
# measure.requires_grad()
# len(measure) == measure.size(0)

TypeError: 'bool' object is not callable

In [27]:
# sigma_target_sq = torch.randn(4, 1, 64, 64)
# g = torch.randn(4, 1, 64, 64)
g *= sigma_target_sq >= EPS
# g[sigma_target_sq < EPS] = 0
g
# g.where(sigma_target_sq >= 0, torch.zeros_like(g)).shape

tensor([[[[ 0.0000,  0.0000,  2.0237,  ..., -0.6767, -1.2737,  0.0000],
          [ 0.0000, -0.5694, -0.9300,  ...,  0.0000,  0.0000, -0.6459],
          [-0.3101,  1.1049,  0.0000,  ...,  0.0000,  0.0000,  1.1628],
          ...,
          [ 0.9193,  0.0000,  0.0000,  ..., -2.0906,  0.0000,  0.0000],
          [ 0.4722,  0.0250,  0.0000,  ..., -0.4501, -0.9140,  0.0000],
          [ 0.0000,  0.0000,  0.5370,  ..., -0.8256, -0.3342,  0.4103]]],


        [[[ 0.0000, -0.2095,  0.1179,  ...,  0.0000,  0.0000, -0.8552],
          [ 0.0000,  0.0000,  0.0000,  ..., -0.8735, -0.5826, -0.0311],
          [-0.1037,  0.9752,  0.2611,  ...,  0.0000,  0.2897, -0.3917],
          ...,
          [ 0.0000, -0.3465,  0.0000,  ...,  0.0000, -0.3884,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ..., -0.0795,  0.0000,  0.0000],
          [-1.0346,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.4758]]],


        [[[-0.0683,  0.0000,  0.0000,  ...,  0.8636,  0.0000,  0.0000],
          [ 0.0000,  0.000

# Imports

In [1]:
# General imports
import os
import sys
import random 
import functools

import cv2
import numpy as np
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt 
import albumentations as albu
import albumentations.pytorch as albu_pt
import photosynthesis_metrics as pm

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

# Fix to be able to import python modules inside a notebook
os.chdir('..')

# Useful extensions
%load_ext watermark
%watermark -v -n -m -p numpy,torch,albumentations,photosynthesis_metrics

# %load_ext autoreload
# %autoreload 2

# Nice plot formating
%matplotlib inline

Fri Jun 12 2020 

CPython 3.6.9
IPython 7.8.0

numpy 1.17.0
torch 1.5.0
albumentations 0.4.5
photosynthesis_metrics 0.4.0

compiler   : GCC 8.4.0
system     : Linux
release    : 4.15.0-99-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 16
interpreter: 64bit


# Сompute metric predictions for TID2013 

1. Prediction in form 1, 121, 241, ...
2. want to be 1, 2, 3, 4, 5, 6, 

In [3]:
import functools
from collections import defaultdict

from torch.nn.modules.loss import _Loss

from src.modules.models import Regression
from src.data.datasets import TID2013
from src.data.utils import crop_patches
from src.modules.losses import ContentLoss, StyleLoss

In [4]:
import torch

class PSNR(_Loss):
    def forward(self, prediction, target):
        mse = torch.mean((prediction - target) ** 2, dim=[1,2,3])
        psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))
        return psnr



In [5]:
# Init Full Reference metrics
psnr = PSNR()
ssim = functools.partial(pm.ssim, data_range=1.0, size_average=False, full=False)
ms_ssim = functools.partial(pm.multi_scale_ssim, data_range=1.0, size_average=False)
vif_p = functools.partial(pm.vif_p, data_range=1.0)
gmsd = pm.GMSDLoss(reduction='none', data_range=1.0)
ms_gmsd = pm.MultiScaleGMSDLoss(reduction='none', data_range=1.0)

# style = StyleLoss(reduction='none')
# content = ContentLoss(reduction='none')

# Init No Reference metrics
brisque = pm.BRISQUELoss(reduction='none')

# Init Distribution based metrics
fid = pm.FID()
kid = pm.KID()
# gs = pm.GS()
isc = pm.IS()

In [7]:
# Init dataset
transform = albu.Compose([
    albu.Normalize(mean=[0., 0., 0.], std=[1., 1., 1.], ), # to [0, 1]
    albu_pt.ToTensorV2(),
])

dataset = TID2013(transform=transform)

# Iterate over dataset
metric_scores = defaultdict(list)
for idx in tqdm(range(len(dataset))):
    # Load images
    distorted_images, reference_images, _ = dataset[idx]
    distorted_images, reference_images = distorted_images.to("cuda"), reference_images.to("cuda")
    
    # Compute metrics
    metric_scores['psnr'].append(psnr(distorted_images, reference_images))
    metric_scores['ssim'].append(ssim(distorted_images, reference_images))
    metric_scores['ms_ssim'].append(ms_ssim(distorted_images, reference_images))
    metric_scores['vif_p'].append(vif_p(distorted_images, reference_images))
    metric_scores['gmsd'].append(gmsd(distorted_images, reference_images))
    metric_scores['ms_gmsd'].append(ms_gmsd(distorted_images, reference_images))
    metric_scores['brisque'].append(brisque(distorted_images))
    
#     with torch.no_grad():
#         metric_scores['style'].append(style(distorted_images, reference_images))
#         metric_scores['content'].append(content(distorted_images, reference_images))
    break

    


In [ ]:
from photosynthesis_metrics.feature_extractors.fid_inception import InceptionV3
distorted_patches = crop_patches(distorted_images)
feature_extractor = InceptionV3(use_fid_inception=False, normalize_input=True)

In [16]:
BATCH_SIZE = 125

In [18]:
# distorted_patches.shape
loader = distorted_patches.view(-1, BATCH_SIZE, *distorted_patches.shape[-3:])
for distorted_patche in loader:
    print(distorted_patche.shape)
    break

torch.Size([125, 3, 64, 64])


In [11]:
features = feature_extractor(distorted_patches)

RuntimeError: CUDA out of memory. Tried to allocate 4.12 GiB (GPU 0; 31.75 GiB total capacity; 8.54 GiB already allocated; 2.90 GiB free; 8.55 GiB reserved in total by PyTorch)

In [ ]:
# Re-init dataset with different normalization
transform = albu.Compose([
    albu.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), # to [-1, 1]
    albu_pt.ToTensorV2(),
])

dataset = TID2013(transform=transform)

# Iterate over dataset
for idx in tqdm(range(len(dataset))):
    # Load images
    distorted_images, reference_images, _ = dataset[idx]
    
    # Create patches 
    distorted_patches = crop_patches(distorted_images)
    reference_patches = crop_patches(reference_images)
    
    # Compute features
    
    fid_score = 
    metric_scores['fid'].append(fid(distorted_images, reference_images))
    predicted_scores = metric(distorted_images, reference_images)
    metric_scores.append(predicted_scores)

In [14]:
for key in metric_scores.keys():
    value = metric_scores[key]
    # Reduce
    scores = torch.stack(value).t().flatten()
    print(key, scores.shape)
    
    # Save to file
    with open(f"data/interim/{key}.txt", "x") as file:
        file.write('\n'.join(str(score) for score in scores.cpu().numpy()))

torch.Size([3000])
torch.Size([3000])
torch.Size([3000])
torch.Size([3000])
torch.Size([3000])
torch.Size([3000])


In [ ]:
# Save result to file
with open("data/interim/ssim.txt", "x") as file:
    file.write('\n'.join(str(score) for score in metric_scores))
    

In [ ]:
metrics = {
    "ssim": pm.ssim,
    "ms-ssim": pm.multi_scale_ssim,
    "vif": pm.vif_p,
    "gmsd": pm.GMSDLoss,
    "ms-gmsd": pm.MultiScaleGMSDLoss,
    "msid": pm.MSID,

}

In [ ]:
r = [torch.rand(4), torch.rand(4), torch.rand(4)]
r

In [ ]:
torch.stack(r).t().flatten()

In [4]:
# MOS scores ordered by image name
file_path = "data/raw/tid2013/mos.txt"
# Read file mith MOS
with open(file_path) as f:
    mos_scores = f.readlines()
mos_scores = [float(score) for score in mos_scores]

# Logistic regression

In [ ]:
import numpy as np
import scipy.stats
from scipy.stats import pearsonr, spearmanr, kendalltau

x = np.arange(10, 20)
y = np.array([2, 1, 4, 5, 8, 12, 18, 25, 96, 48])
x_t = torch.tensor(x, dtype=torch.float32)
y_t = torch.tensor(y, dtype=torch.float32)

r = np.corrcoef(x, y)[0, 1]
r_t = pearson_correlation(x_t, y_t)
print(r, r_t)

print(scipy.stats.pearsonr(x, y)[0])    # Pearson's r

print(scipy.stats.spearmanr(x, y)[0])   # Spearman's rho

print(scipy.stats.kendalltau(x, y)[0])  # Kendall's tau



In [ ]:
# 1. Load MOS score
# 2. Load score for FSIMc
# 3. Fit regression using MSE loss
# 4. Fit regression using PLCC loss
# 5. Compute PLCC, SRCC in both cases
# 6. Compare with values from the paper

learningRate = 0.01
epochs = 10000
    
metric_names = ["PSNRHVSM", "VIFP", "MSSIM", 
#                 "WSNR", "PSNRHMA", "FSIM", "SSIM", "PSNRc", "PSNR", "FSIMc", 
#                "PSNRHVS", "PSNRHA", "VSNR", "NQM"
               ]


for metric in metric_names:
    done = False
    metric_path = "data/raw/tid2013/metrics_values/" + metric + ".txt"
    # Read metric score
    with open(metric_path) as f:
        metric_scores = f.readlines()

    metric_scores = [float(score) for score in metric_scores]

    model = Regression()
#     criterion = torch.nn.MSELoss() 
    criterion = functools.partial(pearson_correlation, invert=True)
    optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)
#     optimizer = torch.optim.AdamW(model.parameters(), lr=learningRate)
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=100, cooldown=50)
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=learningRate, max_lr=0.05)

    prediction = torch.tensor(metric_scores)
    target = torch.tensor(mos_scores)

    # Fit regression
    for epoch in range(epochs + 1):
        if done:
            continue
        # Clear gradient 
        optimizer.zero_grad()

        # get output from the model, given the inputs
        outputs = model(prediction)

        # Get loss for the predicted output
        loss = criterion(outputs, target)
        if torch.isnan(loss):
            print("NaN in loss!")
            done = True
            continue
            
        # get gradients w.r.t to parameters
        loss.backward()

        # update parameters
        optimizer.step()
        scheduler.step(loss)
        if epoch % 5000 == 0:
            print(f'{metric} ({epoch}): {loss.item()}')
    
    if done:
        continue
    # Compute metrics:
    x = outputs.detach().numpy()
    y = mos_scores
    print(f"{metric}: PLCC {pearsonr(x, y)[0]:0.3f}, SRCC {spearmanr(x, y)[0]:0.3f}", 
          f"KRCC {kendalltau(x, y)[0]:0.3f}")

In [ ]:
lines = """VIFP: PLCC 0.752 SRCC 0.658 KRCC 0.481
MSSIM PLCC 0.831 SRCC 0.787 KRCC 0.608
PSNRHMA PLCC 0.799 SRCC 0.813 KRCC 0.632
FSIM PLCC 0.854 SRCC 0.801 KRCC 0.630
SSIM PLCC 0.684 SRCC 0.637 KRCC 0.464
PSNRc PLCC 0.660 SRCC 0.687 KRCC 0.496
FSIMc PLCC 0.870 SRCC 0.851 KRCC 0.667
PSNRHA PLCC 0.796 SRCC 0.819 KRCC 0.643"""
result = {}
plcc = []
srcc = []
krcc = []
for line in lines.split("\n"):
    split = line.split(" ")
    plcc.append((split[0], float(split[2])))
    srcc.append((split[0], float(split[4])))
    krcc.append((split[0], float(split[6])))
    
    result[split[0]] = (float(split[2]), float(split[4]), float(split[6]))
    
result
idx = 1
print("Посчитанные мной")
for pair in sorted(plcc, key = lambda x: x[1], reverse=True):
    print(idx, pair[0], pair[1])
    idx += 1
    

In [ ]:
Данные из статьи
1  FSIMc   0.851        
 2  PSNR-HA 0.819
 3  PSNR-HMA 0.813        
 4  FSIM 0.801     
 5  MSSIM   0.787         
 6  PSNRc   0.687        
 7 SSIM 0.637                                        
 8 VIFP 0.608                     

In [ ]:
# Plot graph: Ox - metric value, Oy - MOS. Blue cross - image in dataset. Black line - fitted logregression
plt.plot(metric_scores, mos_scores, "+")
x = np.arange(0.0, 1.0, 0.005)
y = model(torch.tensor(x))
plt.plot(x, y.detach().numpy())

In [ ]:
FSIMc
MSE
epoch 9000, loss 0.4084590971469879
tensor(0.8559) 

PLCC
epoch 10000, loss 0.14436256885528564
tensor(0.8556)

VIFP
PLCC
tensor(0.7530)

MSE
epoch1000 loss 0.6762036681175232
tensor(0.7519)

# Predict PM lib scores, fit regression, get ranking

# VIF (debug)

In [72]:
r"""Implemetation of Visual Information Fidelity metric
Code is based on MATLAB version for computations in pixel domain
https://live.ece.utexas.edu/research/Quality/VIF.htm

References:
    https://ieeexplore.ieee.org/abstract/document/1576816/
"""
import torch
from torch.nn.modules.loss import _Loss
import torch.nn.functional as F
from typing import Union

from photosynthesis_metrics.utils import _adjust_dimensions, _validate_input


def _gaussian_kernel2d(kernel_size: int = 5, sigma: float = 2.0) -> torch.Tensor:
    r"""Returns 2D Gaussian kernel N(0,`sigma`)
    Args:
        kernel_size: Size
        sigma: Sigma
    Returns:
        gaussian_kernel: 2D kernel with shape (kernel_size x kernel_size)
        
    """
    x = torch.arange(- (kernel_size // 2), kernel_size // 2 + 1).resize(1, kernel_size)
    y = torch.arange(- (kernel_size // 2), kernel_size // 2 + 1).resize(kernel_size, 1)
    kernel = torch.exp(-(x * x + y * y) / (2.0 * sigma ** 2))
    # Normalize
    kernel = kernel / torch.sum(kernel)
    return kernel



with torch.autograd.set_detect_anomaly(True):
    
    x = torch.randn((4, 3, 128, 128), requires_grad=True)
    y = torch.randn((4, 3, 128, 128), requires_grad=True)

    prediction = torch.randn((4, 3, 128, 128), requires_grad=True)
    target = torch.randn((4, 3, 128, 128), requires_grad=True)
#     prediction = x.clone() + 2
#     target = y.clone() + 2
    
    sigma_n_sq: float = 2.0
    data_range: int = 1.0
    reduction: str = 'mean'

    _validate_input((prediction, target), allow_5d=False)
    if data_range == 255:
        prediction = prediction / 255.
        target = target / 255.

    # Convert RGB image to YCbCr and take luminance: Y = 0.299 R + 0.587 G + 0.114 B
    num_channels = prediction.size(1)
#     if num_channels == 3:
#         Y_weights = torch.tensor([[0.299, 0.587, 0.114]]).t().to(prediction)
#         print(prediction.shape, target.shape, Y_weights.shape)
#         prediction = torch.matmul(prediction.permute(0, 2, 3, 1), Y_weights).permute(0, 3, 1, 2)
#         target = torch.matmul(target.permute(0, 2, 3, 1), Y_weights).permute(0, 3, 1, 2)
#         print(prediction.shape, target.shape)

    if num_channels == 3:
        prediction = 0.299 * prediction[:, 0, :, :].clone() + 0.587 * prediction[:, 1, :, :].clone() + 0.114 * prediction[:, 2, :, :].clone()
        target = 0.299 * target[:, 0, :, :].clone() + 0.587 * target[:, 1, :, :].clone() + 0.114 * target[:, 2, :, :].clone()

        # Add channel dimension
        prediction = prediction[:, None, :, :].clone()
        target = target[:, None, :, :].clone()


    # Constant for numerical stability
    EPS = 1e-8
    
    s = prediction.mean() + target.mean()
    s.backward()
    print(s, x.grad)


tensor(-0.0007, grad_fn=<AddBackward0>) None


In [70]:
#     # Progressively downsample images and compute VIF on different scales
#     prediction_vif, target_vif = 0, 0
#     for scale in range(1, 5):
#         kernel_size = 2 ** (5 - scale) + 1
#         kernel = _gaussian_kernel2d(kernel_size, sigma=kernel_size / 5)
#         kernel = kernel.view(1, 1, kernel_size, kernel_size).to(prediction)

#         if scale > 1:
#             # Convolve and downsample
#             prediction = F.conv2d(prediction, kernel)[:, :, ::2, ::2].clone()  # valid padding
#             target = F.conv2d(target, kernel)[:, :, ::2, ::2].clone()  # valid padding

#         mu_trgt, mu_pred = F.conv2d(target, kernel), F.conv2d(prediction, kernel)  # valid padding
#         mu_trgt_sq, mu_pred_sq, mu_trgt_pred = mu_trgt * mu_trgt, mu_pred * mu_pred, mu_trgt * mu_pred

#         sigma_trgt_sq = F.conv2d(target ** 2, kernel) - mu_trgt_sq
#         sigma_pred_sq = F.conv2d(prediction ** 2, kernel) - mu_pred_sq
#         sigma_trgt_pred = F.conv2d(target * prediction, kernel) - mu_trgt_pred
        
#         # Zero small negative values
#         torch.relu_(sigma_trgt_sq)
#         torch.relu_(sigma_pred_sq)

#         g = sigma_trgt_pred / (sigma_trgt_sq + EPS)
#         sigma_v_sq = sigma_pred_sq - g * sigma_trgt_pred

# #         g[sigma_trgt_sq < EPS] = 0
# #         sigma_v_sq[sigma_trgt_sq < EPS] = sigma_pred_sq[sigma_trgt_sq < EPS]
# #         sigma_trgt_sq[sigma_trgt_sq < EPS] = 0

# #         g[sigma_pred_sq < EPS] = 0
# #         sigma_v_sq[sigma_pred_sq < EPS] = 0

# #         sigma_v_sq[g < 0] = sigma_pred_sq[g < 0]
#         g = torch.relu(g)
#         sigma_v_sq[sigma_v_sq <= EPS] = EPS
    
    
        mask_sigma_trgt_sq = sigma_trgt_sq < EPS
        g[mask_sigma_trgt_sq] = 0
        sigma_v_sq[mask_sigma_trgt_sq] = sigma_pred_sq[mask_sigma_trgt_sq]
        sigma_trgt_sq[mask_sigma_trgt_sq] = 0

        mask_sigma_pred_sq = sigma_pred_sq < EPS
        g[mask_sigma_pred_sq] = 0
        sigma_v_sq[mask_sigma_pred_sq] = 0

        mask_g = g < 0
        sigma_v_sq[mask_g] = sigma_pred_sq[mask_g]
        g = torch.relu(g)
        mask_sigma_v_sq = sigma_v_sq <= EPS
        sigma_v_sq[mask_sigma_v_sq] = EPS
        
#         pred_vif_scale = torch.log10(1.0 + (g ** 2.) * sigma_trgt_sq / (sigma_v_sq + sigma_n_sq))
#         prediction_vif = prediction_vif + torch.sum(pred_vif_scale, dim=[1, 2, 3])
#         target_vif = prediction_vif + torch.sum(torch.log10(1.0 + sigma_trgt_sq / sigma_n_sq), dim=[1, 2, 3])

#     vif = (prediction_vif + EPS) / (target_vif + EPS)
    
#     print(vif)
#     vif.mean(dim=0).backward()
#     print(vif, target.grad)

tensor(0.0028, grad_fn=<AddBackward0>) None


In [44]:
 


#     # Reduce if needed
#     if reduction == 'mean':
#         return vif.mean(dim=0)
#     elif reduction == 'sum':
#         return vif.sum(dim=0)
#     elif reduction != 'none':
#         raise ValueError(f'Expected reduction modes are "mean"|"sum"|"none", got {reduction}')
#     return vif

RuntimeError: number of dims don't match in permute

In [35]:

    prediction = torch.randn((4, 3, 64, 64), requires_grad=True)
    target = torch.randn((4, 3, 64, 64), requires_grad=True)
    score = vif_p(x, y)
    score.backward()

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [4, 1, 3, 3]], which is output 0 of DivBackward0, is at version 3; expected version 0 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

# Garbage

In [ ]:
class HDRLoss(nn.Module):
    """High dynamic range loss."""

    def __init__(self, eps=0.01):
        """Initializes loss with numerical stability epsilon."""

        super(HDRLoss, self).__init__()
        self._eps = eps


    def forward(self, denoised, target):
        """Computes loss by unpacking render buffer."""

        loss = ((denoised - target) ** 2) / (denoised + self._eps) ** 2
        return loss.mean(dim=[1,2,3])

In [ ]:
"""
Implementation of VGG16 loss, originaly used for style transfer and usefull in many other task (including GAN training)
It's work in progress, no guarantees that code will work
"""
import collections

import torch
import torch.nn as nn
from torch.nn.modules.loss import _Loss
from torchvision.models import vgg16


def listify(p):
    if p is None:
        p = []
    elif not isinstance(p, collections.Iterable):
        p = [p]
    return p


class ContentLoss(_Loss):
    """
    Creates content loss for neural style transfer.
    Uses pretrained VGG16 model from torchvision by default
    layers: list of VGG layers used to evaluate content loss
    criterion: str in ['mse', 'mae'], reduction method
    reduction: Type of reduction to use
    """

    def __init__(
        self,
        layers=["21"],
        weights=1,
        loss="mse",
        device="cuda",
        reduction="mean",
        **args,
    ):
        super().__init__()
        self.model = vgg16(pretrained=True, **args)
        self.model.eval().to(device)
        self.layers = listify(layers)
        self.weights = listify(weights)

        if loss == "mse":
            self.criterion = nn.MSELoss(reduction=reduction)
        elif loss == "mae":
            self.criterion = nn.L1Loss(reduction=reduction)
        else:
            raise KeyError

    def forward(self, input, content):
        """
        Measure distance between feature representations of input and content images
        """
        input_features = torch.stack(self.get_features(input))
        content_features = torch.stack(self.get_features(content))
        loss = self.criterion(input_features, content_features)

        # Solve big memory consumption
        torch.cuda.empty_cache()
        return loss

    def get_features(self, x):
        """
        Extract feature maps from the intermediate layers.
        """
        if self.layers is None:
            self.layers = ["21"]

        features = []
        for name, module in self.model.features._modules.items():
            x = module(x)
            if name in self.layers:
                features.append(x)
        # print(len(features))
        return features


class StyleLoss(_Loss):
    """
    Class for creating style loss for neural style transfer
    model: str in ['vgg16_bn']
    reduction: Type of reduction to use
    """

    def __init__(
        self,
        layers=["0", "5", "10", "19", "28"],
        weights=[0.75, 0.5, 0.2, 0.2, 0.2],
        loss="mse",
        device="cuda",
        reduction="mean",
        **args,
    ):
        super().__init__()
        self.model = vgg16(pretrained=True, **args)
        self.model.eval().to(device)

        self.layers = listify(layers)
        self.weights = listify(weights)

        if loss == "mse":
            self.criterion = nn.MSELoss(reduction=reduction)
        elif loss == "mae":
            self.criterion = nn.L1Loss(reduction=reduction)
        else:
            raise KeyError

    def forward(self, input, style):
        """
        Measure distance between feature representations of input and content images
        """
        input_features = self.get_features(input)
        style_features = self.get_features(style)
        # print(style_features[0].size(), len(style_features))

        input_gram = [self.gram_matrix(x) for x in input_features]
        style_gram = [self.gram_matrix(x) for x in style_features]

        loss = [
            self.criterion(torch.stack(i_g), torch.stack(s_g)) for i_g, s_g in zip(input_gram, style_gram)
        ]
        return torch.mean(torch.tensor(loss))

    def get_features(self, x):
        """
        Extract feature maps from the intermediate layers.
        """
        if self.layers is None:
            self.layers = ["0", "5", "10", "19", "28"]

        features = []
        for name, module in self.model.features._modules.items():
            x = module(x)
            if name in self.layers:
                features.append(x)
        return features

    def gram_matrix(self, input):
        """
        Compute Gram matrix for each image in batch
        input: Tensor of shape BxCxHxW
            B: batch size
            C: channels size
            H&W: spatial size
        """

        B, C, H, W = input.size()
        gram = []
        for i in range(B):
            x = input[i].view(C, H * W)
            gram.append(torch.mm(x, x.t()))
        return gram
